Now let's see how my overload decorator does this, to see if it generates multiple seperate LLVM modules

In [1]:
import numba
import os

In [2]:
os.environ['NUMBA_DUMP_OPTIMIZED'] = '1'

We notice these modules are actually combined in the `there` module. `__main__9there` contains both print statements

In [3]:
import llvmlite

import numba

# a byte
llvm_type = llvmlite.ir.IntType(64)


class TestStructType(numba.types.Type):
    def __init__(self):
        super().__init__(name="TestStruct")


test_struct_type = TestStructType()


@numba.extending.register_model(TestStructType)
class TestStructModel(numba.extending.models.PrimitiveModel):
    def __init__(self, dmm, fe_type):
        super().__init__(dmm, fe_type, llvmlite.ir.PointerType(llvm_type))


@numba.extending.intrinsic
def create_test_struct(typingctx, int_t):

    sig = test_struct_type(int_t)

    def codegen(context, builder, sig, args):
        a = builder.alloca(llvm_type)
        builder.store(value=args[0], ptr=a)
        return a

    return sig, codegen


@numba.extending.intrinsic
def convert_test_struct(typingctx, test_struct_t):

    sig = numba.types.int64(test_struct_t)

    def codegen(context, builder, sig, args):
        return builder.load(args[0])

    return sig, codegen




def overload_any(func):
    """
    Like `numba.extending.overload` but works for things like `getitem`, etc.

    Used likes `generated_jit`:

        @overload_any("getitem")
        def getitem_const(val, i):
            if val.value == "hi":
                return lambda val, i: i
            elif val.value == "there":
                return lambda val, i: -i
    """

    def inner(overload_func):
        # lower dispatcher based on `numba.typing.templates._OverloadMethodTemplate.do_class_init`
        dispatcher = numba.generated_jit(nopython=True)(overload_func)
        disp_type = numba.types.Dispatcher(dispatcher)

        def impl(context, builder, sig, args):
            call = context.get_function(disp_type, sig)
            cg = context.codegen()
            for lib in getattr(call, "libs", ()):
                cg.add_linking_library(lib)
            return call(builder, args)

        @numba.extending.type_callable(func)
        def type_inner(context):
            # need to pass in `dispatcher` or get "underlying object has vanished"
            def typer(*args, dispatcher=dispatcher):
                try:
                    sig = disp_type.get_call_type(context, args, {})
                except TypeError:  # None returned by overloaded function
                    return
                if sig:
                    # ideally, instead of adding a lowering for this specific type, we would just return the `impl`
                    # with the typing so it doesn't have to look it up. I am not sure how to do this in `type_callable`, though.
                    numba.targets.imputils.lower_builtin(func, *sig.args)(impl)
                    return sig.return_type

            return typer

    return inner


@overload_any("getitem")
def getitem_const(val, i):
    def impl(val, i):
        a = create_test_struct(i)
#         print(convert_test_struct(a))
        return a

    return impl


@numba.njit
def second_test():
    s = "hithere"[10]
    print(convert_test_struct(s))


second_test()

-------------------OPTIMIZED DUMP getitem_const.<locals>.impl-------------------
; ModuleID = 'getitem_const.<locals>.impl'
source_filename = "<string>"
target datalayout = "e-m:o-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-apple-darwin17.7.0"

@"_ZN08NumbaEnv8__main__13getitem_const12$3clocals$3e8impl$242E24const$28$27hithere$27$29x" = common local_unnamed_addr global i8* null
@".const.getitem_const.<locals>.impl" = internal constant [28 x i8] c"getitem_const.<locals>.impl\00"
@PyExc_RuntimeError = external global i8
@".const.missing Environment" = internal constant [20 x i8] c"missing Environment\00"
@PyExc_TypeError = external global i8
@".const.can't unbox const('hithere') type" = internal constant [34 x i8] c"can't unbox const('hithere') type\00"

; Function Attrs: norecurse nounwind
define i32 @"_ZN8__main__13getitem_const12$3clocals$3e8impl$242E24const$28$27hithere$27$29x"(i64** noalias nocapture %retptr, { i8*, i32 }** noalias nocapture readnone %excinfo, i8* nocap

Here we can see that the meat of this code, the allocating and storing:

```
  %.18 = alloca i64, align 8
  store i64* %.18, i64** %retptr, align 8
```

actually only shows up in `getitem_const.<locals>.impl` and not in `second_test`. Why is this?

OK this is really weird. Depending on whether I comment out `print(convert_test_struct(a))` or not, it will return the right answer or not. Let's do a new approach where we see how it lowers a funciton it encounters, by jitting that first, and trying to copy that for builtin overloads.